# HW2 Overview

In this assignment, we will study language model. You will get the basic ideas of maximum likelihood estimation, smoothing, generate text documents from language models, and language model evaluation. 

We will reuse the same Yelp dataset and refer to each individual user review as a **document** (e.g., as in computing document frequency). You should reuse your JSON parser in this assignment.

The same pre-processing steps you have developed in HW1 will be used in this assignment, i.e., tokenization, stemming and normalization. Note: **NO** stopword removal is needed in this assignment. 



# Statistical Language Models

### 1. Maximum likelihood estimation for statistical language models with proper smoothing (50pts)

Use all the review documents to estimate a unigram language model $p(w)$ and two bigram language models (with different smoothing methods specified below). Note those language models are corpus-level models, i.e., aggregating all the words across different documents.

When estimating the bigram language models, using linear interpolation smoothing and absolute discount smoothing based on the unigram language model $p_u(w)$ to get two different bigram language models accordingly, i.e., $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$. In linear interpolation smoothing, set the parameter $\lambda=0.9$; and in absolute discount smoothing, set the parameter $\delta=0.1$.

Specifically, when estimating $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$, you should use the unigram language model $p(w_i)$ as the reference language model in smoothing. For example, in linear interpolation smoothing, the resulting smoothing formula looks like this,

$$p^L(w_i|w_{i-1})=(1-\lambda) \frac{c(w_{i-1}w_i)}{c(w_{i-1})} + \lambda p(w_i)$$ 
where $c(w_{i-1}w_i)$ is the frequency of bigram $w_{i-1}w_i$ in the whole corpus.

From the resulting two bigram language models, find the top 10 words that are most likely to follow the word "good", i.e., rank the words in a descending order by $p^L(w|good")$ and $p^A(w|good")$ and output the top 10 words. Are those top 10 words the same from these two bigram language models? Explain your observation.

*HINT: to reduce space complexity, you do not need to actually maintain a $V\times V$ array to store the counts and probabilities for the bigram language models. You can use a sparse data structure, e.g., hash map, to store the seen words/bigrams, and perform the smoothing on the fly, i.e., evoke some function calls to return the value of $p^L(w|good")$ and $p^A(w|good")$.* 

**What to submit**:

1. Paste your implementation of the linear interpolation smoothing and absolute discount smoothing.
2. The top 10 words selected from the corresponding two bigram language models.
3. Your explanation of the observations about the top words under those two bigram language models.



In [1]:
from utils import *
import pickle

In [2]:
#unigram language model p(w)
list_of_tokenized_reviews = []
with open('list_of_tokenized_reviews.pickle', 'rb') as file:
    list_of_tokenized_reviews = pickle.load(file)
token_freq_dict = total_term_frequency(list_of_tokenized_reviews)
total_num_tokens = sum([len(tokens) for tokens in list_of_tokenized_reviews])
token_prob_dict = {}
for key in token_freq_dict.keys():
    token_prob_dict[key] = token_freq_dict[key] / total_num_tokens


In [3]:
#bigram language model with linear interpolation smoothing
all_bigrams = get_all_bigrams(list_of_tokenized_reviews)
bigram_freq_dict = total_bigram_freqency(all_bigrams)
def p_linear(first, second):
    lam = 0.9
    try:
        prob = (1-lam) * (bigram_freq_dict[(first, second)] / token_freq_dict[first]) + (lam * token_prob_dict[second])
    except:
        prob = (lam * token_prob_dict[second])
    return prob
#get the top most likely words to follow "good"
linear_probabilities = {}
for token in token_prob_dict.keys():
    linear_probabilities[token] = p_linear('good', token)
ans = dict(sorted(linear_probabilities.items(), key = lambda x: x[1], reverse = True)[:10])
for line in ans:
    print(f'{line} : {ans[line]}')    

the : 0.05306509260194806
and : 0.03497929899421944
i : 0.03195698201920177
a : 0.02365215008387146
to : 0.019990190189529227
but : 0.01815881364406983
it : 0.017903926404976605
wa : 0.01704291139500162
of : 0.014552406250610416
for : 0.011796944880580106


In [4]:
#most popular words to follow 'good' with no smooting.
target_bigrams = {}
for bigram in bigram_freq_dict.keys():
    if bigram[0] == 'good':
        target_bigrams[bigram] = bigram_freq_dict[bigram]
ans = dict(sorted(target_bigrams.items(), key = lambda x: x[1], reverse = True)[:10])
for line in ans:
    print(f'{line} : {ans[line]}')

('good', 'but') : 7560
('good', 'and') : 5022
('good', 'i') : 4455
('good', 'the') : 4358
('good', 'as') : 2874
('good', 'food') : 2306
('good', 'it') : 1507
('good', 'thing') : 1403
('good', 'for') : 1393
('good', 'too') : 1385


In [5]:
#bigram language model with absolute discount smoothing
def p_abs_disc(first, second):
    delta = 0.1
    d_u = len(target_bigrams) #number of unique bigrams with 'good' as the first word
    try:
        prob = (max(bigram_freq_dict[(first, second)] - delta, 0) + (delta * d_u * token_prob_dict[second])) / (token_freq_dict[first])
    except:
        prob = (delta * d_u * token_prob_dict[second]) / (token_freq_dict[first])
    return prob

abs_disc_probabilities = {}
for token in token_prob_dict.keys():
    abs_disc_probabilities[token] = p_abs_disc('good', token)
ans = dict(sorted(abs_disc_probabilities.items(), key = lambda x: x[1], reverse = True)[:10])
for line in ans:
    print(f'{line} : {ans[line]}')

but : 0.09564301951999492
and : 0.063624517412393
i : 0.05644462810001931
the : 0.05530186422700514
as : 0.03636073506188902
food : 0.029181046543558607
it : 0.019120838648309255
thing : 0.017747196904450483
for : 0.01765550959445897
too : 0.01752179260050426


### 2. Generate text documents from a language model (40pts)

Fixing the document length to 20, generate 10 documents by sampling words from $p(w)$, $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$ respectively.

*HINT: you can use $p(w)$ to generate the first word of a document and then sampling from the corresponding bigram language model when generating from $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$.* 

**What to submit**:

1. Paste your implementation of the sampling procedure from a language model.
2. The 10 documents generated from $p(w)$, $p^L(w_i|w_{i-1})$ and $p^A(w_i|w_{i-1})$ accordingly, and the corresponding likelihood given by the used language model.

In [6]:
import numpy as np
doc_size = 20

In [7]:
#unigram
unigram_tokens = []
unigram_probs = []
for key in token_prob_dict.keys():
    unigram_tokens.append(key)
    unigram_probs.append(token_prob_dict[key])
for i in range(10):
    samples = np.random.choice(unigram_tokens, doc_size, p=unigram_probs)
    final_probability = token_prob_dict[samples[0]]
    for i in range(1, len(samples)):
        final_probability *= token_prob_dict[samples[i]]
    print(' '.join(samples)+ ' : ' + str(final_probability))

belli like becam i with also year someth sort spend it them dish wa student fantast we wo you is : 3.203829260978685e-58
sinc there geog offer and dine better skip quit i hand burger wa the the broccoli into throw the pizza : 9.362810067602832e-60
beef the around is up okay sport it neighborhood brought short tasti give onc part it angel did ve pack : 9.409861206274809e-61
place becaus bar of or compens good lox the bar good and go sure idiot main along skirt with batch : 4.8775442722828e-62
i sweetheart no think salad roll of that waitress food man flash companion a one i there serv be the : 1.7356431567417256e-56
tini strictli littl it it no sport love me experi seri my with our delici to you those differ french : 2.764208419580971e-59
the portion definit enough the him the by can miss get the would that me a chicken pizza mainten it : 1.1491056590870506e-51
it on side st we big the here place sunday which to of previou mani definit for look had becaus : 9.92993180874294e-53
henc aro

In [11]:
#bigram linear interp

for i in range(10):
    samples = []
    prev_word = np.random.choice(unigram_tokens, 1, p=unigram_probs)[0]
    samples.append(prev_word)
    final_probability = token_prob_dict[prev_word]
    for i in range(doc_size-1):
        lin_int_probs_dict = {}
        for token in token_prob_dict.keys():
            lin_int_probs_dict[token] = p_linear(prev_word, token)
        lin_int_tokens = []
        lin_int_probs = []
        for key in lin_int_probs_dict.keys():
            lin_int_tokens.append(key)
            lin_int_probs.append(lin_int_probs_dict[key])
        lin_int_probs = np.array(lin_int_probs)
        lin_int_probs /= sum(lin_int_probs)
        prev_word = np.random.choice(lin_int_tokens, 1, p=lin_int_probs)[0]

        samples.append(prev_word)
        final_probability *= lin_int_probs_dict[prev_word]
        
    print(' '.join(samples) + ' : ' + str(final_probability))
    


appet woodi s ok the the would is while flavor the is due just sure find ani s the rosco : 9.423746067339852e-54
both am dine thi look for crust salmon themselv and is of it the thi how crowd go came a : 3.325664595975003e-51
the ve mind team eaten hype than veri flavori great notch also nt as into then so a restaur a : 3.618785728977104e-60
a sandwich and loiter cobbler the a someplac her and two the over as their will tabl calamari nt wa : 2.6663383519321672e-55
we all you can you wa beer whi in thi nice great food bit wa place of atmospher 30pm beef : 2.5805764312122374e-50
campbel wa tast wa veri were not is sweet better for nt disappoint i hear food but if drum the : 6.672544242351564e-53
as the dessert tast perfectli is select hot take good can the my and tabl they pretti the tongu to : 2.5363192214949507e-50
lo squid a also hypnot lesson fish when wait plate spanish and made and the would prefer beehiv not are : 2.4111723100255136e-62
urin kitschi milk mayb first nt the e tasti 

In [12]:
#bigram absolute discount
for i in range(10):
    samples = []
    prev_word = np.random.choice(unigram_tokens, 1, p=unigram_probs)[0]
    samples.append(prev_word)
    final_probability = token_prob_dict[prev_word]
    for i in range(doc_size-1):
        abs_disc_probs_dict = {}
        for token in token_prob_dict.keys():
            abs_disc_probs_dict[token] = p_abs_disc(prev_word, token)
        abs_disc_tokens = []
        abs_disc_probs = []
        for key in abs_disc_probs_dict.keys():
            abs_disc_tokens.append(key)
            abs_disc_probs.append(abs_disc_probs_dict[key])
        abs_disc_probs = np.array(abs_disc_probs)
        abs_disc_probs /= sum(abs_disc_probs)
        prev_word = np.random.choice(abs_disc_tokens, 1, p=abs_disc_probs)[0]
        samples.append(prev_word)
        final_probability *= abs_disc_probs_dict[prev_word]
    print(' '.join(samples) + ' : ' + str(final_probability))

oliv oil yum kun food and you ve been disappoint articl stupend ca nt big and the mussel food prep : 1.4779190893089667e-47
NUM pitcher or i enjoy the front of brisket is incred disappoint dippin pick bachi burger well season and realli : 3.036954549756277e-43
breaker diminish the wait NUM NUM star for NUM month later we start on the parmesan crust overal review fantast : 2.877254956175923e-43
empti threat price for an old pie but nope that the wall sign leg say except amount of the artichok : 5.997603303385795e-51
from the squash ravioli and follow the latk failb franki a night wa nt come back to eat where you : 1.7073004860111123e-39
patron a young to becom a lot of thing to expect had pizza wa pretti fun to go to eat : 1.28968800792104e-41
be a scale goe my recommend from what time they are much but the eggplant anoth parti or simpli delici : 1.8326197847967852e-44
look over ambianc fantast swank of them blah blah quit small the food to go away thi a wide eye : 5.131260330517393e-47

# Reading Assignment — Belief or Bias in Information Retrieval (10pts)
In our class, we have learned both classical and modern information retrieval evaluation methods. And their shared goal is to assess if a retrieval system can satisfy users' information need. Such an evaluation directly leads to the subsequent optimization of retrieval system, e.g., optimize the ranking for click-through rates. But should a system please its users so as to improve the metrics or should it educate the users about what is right and wrong?

Let's read the paper ["Beliefs and biases in web search"](https://dl.acm.org/doi/10.1145/2484028.2484053), which is the best paper in SIGIR'2013. Based on the findings of this paper and current public concern/debate of the wide spread of misinformation on the web, what kind of suggestion do you want to give to Google and Bing to improve the situation? You can focus on the search evaluation, document retrieval and ranking, or any aspect related to the retrieval process.

# Extra Credits (5pts)

You are encouraged to further investigate the relation between classic language model and the trending Large Language Models. How LLMs differ from unigram and bigram models we implemented? It is okay to consult LLMs for this question :\) 

# Submission

This assignment has in total 100 points. The deadline is Feb 20 23:59 PDT. You should submit your report in **PDF** using the homework latex template, and submit your code (notebook)。